In [1]:
import matlab.engine
import numpy as np
import scipy.optimize

In [17]:
a = np.array(range(10))
np.argwhere(a>5)[0]

array([6])

In [18]:
eng = matlab.engine.start_matlab()
eng.cd(r'~/develop/withinhostHIV/MatlabCode/', nargout=0)
eng.eval("addpath('Parameters','Utilities')", nargout=0)

In [27]:
drug = "ATV/r"

days = np.double(40) #days to run trial for
increment = 0.01 # days between each timepoint returned
prob_adh = 0.5 if drug != 'TEST' else 0. # probability of taking each pill (independent)
adh_pat = 0 # 0 = random prob for each dose, 1 = random prob each day after missed dose
adh_shuff = np.double(0.) # standard deviation in the time of taking a pill vs scheduled (days)
trial_type = 1 # 1 = suppression trial, 0=maintenance trial
burnin = 7*5 # days before interval sampling begins in maintenance trials
sampling_int = 28 # days between each sampling time in maintenance trials
threshold = 200 # threshold viral load above which failure is declared
mut_frac = 0.2 # threshold fraction of population of mutant above which declared failure via resistance

# trial and pd parameters

In [28]:
eng.workspace["drug"] = drug
trial, pd = eng.trial_setup(eng.eval("drug"), days, increment, prob_adh, adh_pat, adh_shuff, trial_type, burnin, sampling_int, threshold, mut_frac, nargout=2)
eng.workspace["trial"] = trial
eng.workspace["pd"] = pd

In [29]:
dose_t, _ = eng.adh_trajectory(eng.eval("pd.num_doses"), trial, nargout=2)

In [30]:
c_vec,inhib_vec = eng.drug_trajectory(pd, trial, dose_t, nargout=2)

In [31]:
smin=0.05; #highest mutant fitness
smax=0.9; #lowest mutant fitness
smiss=0.05; #fitness of strains with missing cost
rfmiss=1; #fold change in resistance for strains with it missing
mfmiss=0; #fractional change in slope for strains with it missing
back_mutation_on = drug != 'TEST' #include(1) or exclude(0) back mutation
direct_multi_hit = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from WT
direct_multi_multi = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from one res strain to another

In [32]:
mparams,Q = eng.getMutInfo(drug,smin,smax,smiss,rfmiss,mfmiss,back_mutation_on,direct_multi_hit,direct_multi_multi,nargout=2);
eng.workspace["Q"] = Q
Q = eng.eval("Q{:,:};") # get a matrix instead of a matlab table

## viral dyanmics parameters

In [43]:
R00 = 10.0 # average number of new virions produced by infection with a single virion === average number of new infected cells produced by 1 infected cell
fbp = 0.55 # fraction of blood that is plasma
Vpl = 3*10e3 # volume of plasma in mL
ftcell_pl = 0.02 # fraction of T cells that are circulating in blood (vs in lymph system)
hl_lr = 44*30.5 # half life of decay of latent reservoir, days
A = 100 # total reactivation of latent cells per day
#### TODO CHECK CHECK CHECK SCIENTIFIC NOTATION
flr = 1e-6# fraction of CD4 T cells that are latently infected at equilibrium

scale_cd4_body = (Vpl*10**3)/(fbp*ftcell_pl) # factor to go from T cell per ul blood to whole body

fa = 0.01 # fraction of infected CD4 cells that support productive vs abortive infection
dx = 0.05 # death rate of uninfected cells (per day) and by assumption dx == d, rate of death without viral cytolytic effects
L = scale_cd4_body*1000*dx # uninfected cells produced per day (/ul)
a = A/(flr*L/dx) # rate of exit of latent cells (per day)
dz = np.log(2)/(hl_lr)-a # death rate of latently infected cells (per day)

dy = 1 # TOTAL death rate of infected cells (per day) (=death due to burst (k) + death without viral cytolytic effects)
k = dy-dx # rate of death+emission of a burst of virions (per day)
# CHECK SCI NOTO
N = 2.38e5
assert(N > R00);

# probability of a single virion establishing infection, solved implicitly
p_est_solution = scipy.optimize.least_squares(lambda p_est: R00*(1-(1-p_est)**N) - N*p_est, 0.1)
assert(p_est_solution.success)
p_est = p_est_solution.x

dv = 25 # TOTAL death rate of virions (per day)
beta = R00 * dy * dv * dx / (k * N * fa * L) # infectivity rate (per day*infectious-target cell pair)
c = dv-beta*L/dx # clearance rate of virions (per day);
assert(c > 0)
g = flr*dy/dx*(a+dz)/(fa*(1-1/R00)) # fraction of new infections that become latent



In [82]:
eng.workspace["mparams"] = mparams
t_vec = eng.eval('(0:trial.increment:trial.days);')

IC50 = eng.eval('pd.IC50')
m = eng.eval('pd.m')
cost = eng.eval('mparams.cost')
rf = eng.eval('mparams.rf')
mf = eng.eval('mparams.mf')

beta_t, beta_u_t = eng.calculate_beta_of_t(t_vec, beta, c_vec, IC50, m, cost, rf, mf, eng.eval('height(mparams)'), nargout=2)

In [54]:
beta_t.size

(4001, 1)

In [52]:
beta_u_t.size

(4001, 3)

matlab.double([[1.806403393237949e-19,3.448531637971857e-20,1.3482643523300314e-18],[1.877045567912034e-19,3.583393590541555e-20,1.4034321078051082e-18],[1.950450259988659e-19,3.7235295713891077e-20,1.4608570154217138e-18],[2.0267254956287042e-19,3.869145829082854e-20,1.5206314099830369e-18],[2.1059835249366017e-19,4.0204566776709687e-20,1.5828514022332991e-18],[2.18834098709429e-19,4.1776848120734635e-20,1.6476170331659837e-18],[2.2739190819488783e-19,4.3410616358061503e-20,1.7150324346292631e-18],[2.3628437483060976e-19,4.510827601518717e-20,1.785205996484889e-18],[2.4552458491913867e-19,4.687232564847784e-20,1.8582505405871403e-18],[2.5512613643507346e-19,4.870536152105533e-20,1.9342835018592425e-18],[2.6510315902739545e-19,5.0610081423446764e-20,2.0134271167558528e-18],[2.754703348034152e-19,5.258928864361863e-20,2.09580861941189e-18],[2.862429199248522e-19,5.464589609223351e-20,2.1815604457900826e-18],[2.9743676704775762e-19,5.678293058919825e-20,2.270820446152233e-18],[3.09068348

In [92]:
"""
Bwt_t=B./(1+(c_vec./IC50).^m);
Bu_t_mat=(B./repmat(cost',length(t_vec),1))./(1+(repmat(c_vec,1,num_muts)./(IC50*repmat(rf',length(t_vec),1))).^(m*(1+repmat(mf',length(t_vec),1))));
"""

def calculate_beta_t(beta_0, concentration, IC50, m, cost, rf, mf):
    # B/[cost * (1 + concentration)/(IC50*rf)^(m*(1+mf))]
    B = beta_0
    denominator = cost * (1 + (concentration/(IC50*rf))**(m*(1+mf)))
    return B/denominator


# make sure we didn't mess up our function definition
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T), np.asarray(beta_u_t)))
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1), np.asarray(beta_t)))

beta_u_t = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T)
beta_t   = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1)

beta_t = np.concatenate([beta_t, beta_u_t], axis=1)


In [ ]:
def k_of_t(t):
    

In [ ]:
"""
% with dv sepcified (instead of c)
vd.dv=25; % 
vd.B=vd.R00*vd.dy*vd.dv*vd.dx/(vd.k*vd.N*vd.fa*vd.L); % infectivity rate (per day*target cells)
vd.c=vd.dv-vd.B*vd.L/vd.dx;% clearance rate of virions (per day);
assert(vd.c>0)
vd.g=flr*(vd.dy/vd.dx)*(vd.a+vd.dz)/(vd.fa*(1-1/vd.R00)); % fraction of new infections that become latent

kdv=100; %ratio of viral burst size to viral clearance rate. used ONLY for plotting. % THAYER: fix this!
scale_cd4_virus=kdv*2*1e3/fbp; %to go from T cell per ul in blood to virus/ml in plasma
vd.sigma=vd.scale_cd4_body/scale_cd4_virus; %to go from total body infected cells to virus/ml plasma (Y=sigma*v)
"""

# running forward simulation

In [ ]:
# make stoich matrix
# make rate involvement matrix
# make rate constant function that chops up beta over time